In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
from torch.nn import Softplus #smooth relu
import torch.nn as nn
import torch.nn.functional as F
from utils import generate_polynomial_data
import numpy as np
import torch
import torch.optim as optim
from torch.nn import MSELoss
from tqdm import tqdm

from model import Net, check_loss_landscape

In [28]:
torch.manual_seed(0)

suffix = "3"

n = 5000
d = 10000
generate_data = False

coeffs = np.random.rand(d, 1)
xvals = np.random.rand(n)
if generate_data:
    print("Generating Data...")
    X, Y = generate_polynomial_data(coeffs, xvals)
    '''with open("./datasets/X%s.npy" %suffix, "wb") as f:
        np.save(f, X)
    with open("./datasets/Y%s.npy" %suffix, "wb") as f:
        np.save(f, Y)
    with open("./datasets/coeffs%s.npy" %suffix, "wb") as f:
        np.save(f, coeffs)'''
else:
    with open("./datasets/X%s.npy" %suffix, "rb") as f:
        X = np.load(f)
    with open("./datasets/Y%s.npy" %suffix, "rb") as f:
        Y = np.load(f)
    with open("./datasets/coeffs%s.npy" %suffix, "rb") as f:
        coeffs = np.load(f)

In [29]:
net = Net(d, epochs = 20)

In [30]:
device = next(net.parameters()).device

In [31]:
X_cuda = torch.from_numpy(X).to(device)
Y_cuda = torch.from_numpy(Y).to(device)

In [ ]:
net.train_gd(X, Y, T=2000, lr=1e-03)

  0%|          | 1/2000 [00:00<04:18,  7.72it/s]

Epoch 1 iteration 2000 loss: 4009.78


  0%|          | 1/2000 [00:00<04:22,  7.60it/s]

Epoch 2 iteration 2000 loss: 1255.26


  0%|          | 1/2000 [00:00<04:28,  7.44it/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 6 iteration 2000 loss: 1255.26


  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 7 iteration 2000 loss: 1255.26


 53%|█████▎    | 1057/2000 [04:44<04:12,  3.74it/s]

In [ ]:
model_path = "./models/model_%s.pt"%suffix
torch.save(net.state_dict(), model_path)

In [ ]:
check_loss_landscape(model_path, X, Y)